# Southern Ocean Mode index
annual SST anomaly (50S-35S, 0E-50W), here detrended

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import bll_SOM
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D

In [ ]:
from SST_indices import SOM_index

In [ ]:
(blats, blons) = bll_SOM

In [ ]:
%%time
SOM_ctrl = SOM_index('ctrl')
SOM_rcp  = SOM_index('rcp')
SOM_lpi  = SOM_index('lpi')
SOM_lpd  = SOM_index('lpd')
SOM_had  = SOM_index('had')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

# plt.axhline(0,c='k', lw=.5)
plt.plot(SOM_ctrl.time/365     , SOM_ctrl, c='C0')
plt.plot(SOM_rcp .time/365-1800, SOM_rcp , c='C1')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('raw SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2, loc=4, handles=[L1, L2])
# plt.savefig(f'{path_results}/SST/SOM_raw')

In [ ]:
f,ax = plt.subplots(1,1,figsize=(12,5), sharex=True)

ax.tick_params(labelsize=14)
ax.set_xlabel('time [years]', fontsize=16)
    
ax.set_ylabel('SOM area temperature [$^\circ$C]', fontsize=16)
L1, = ax.plot(SOM_ctrl.time/365+1850, SOM_ctrl, lw=.5, ls=':', c='C0', label='yearly data')
ax.plot(      SOM_rcp .time/365+ 200, SOM_rcp , lw=.5, ls=':', c='C1')
ax.plot(      SOM_lpd .time/365+1350, SOM_lpd , lw=.5, ls=':', c='C2')
ax.plot(      SOM_lpi .time/365-1600, SOM_lpi , lw=.5, ls=':', c='C3')
ax.plot(      SOM_had .time/365+2350, SOM_had , lw=.5, ls=':', c='C4')

L2, = ax.plot(SOM_ctrl.time/365+1850, lowpass(SOM_ctrl, 13), c='C0', label='lowpass 13 yr')
ax.plot(      SOM_rcp .time/365+ 200, lowpass(SOM_rcp , 13), c='C1')
ax.plot(      SOM_lpd .time/365+1350, lowpass(SOM_lpd , 13), c='C2')
ax.plot(      SOM_lpi .time/365-1600, lowpass(SOM_lpi , 13), c='C3')
ax.plot(      SOM_had .time/365+2350, lowpass(SOM_had , 13), c='C4')

L3, = ax.plot(SOM_ctrl.time/365+1850, xr_lintrend(SOM_ctrl)+1, c='C0', ls='-.', lw=1.5, label='linear trend (1 K offset)')
ax.plot(      SOM_rcp .time/365+ 200, xr_lintrend(SOM_rcp )+1, c='C1', ls='-.', lw=1.5)
ax.plot(      SOM_lpd .time/365+1350, xr_lintrend(SOM_lpd )+1, c='C2', ls='-.', lw=1.5)
ax.plot(      SOM_lpi .time/365-1600, xr_lintrend(SOM_lpi )+1, c='C3', ls='-.', lw=1.5)
ax.plot(      SOM_had .time/365+2350, xr_lintrend(SOM_had )+1, c='C4', ls='-.', lw=1.5)

ax.legend(handles=[L1,L2,L3], fontsize=14, ncol=3, frameon=False)

ax.text(1950, 12.3, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, 12.3, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, 12.3, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, 12.3, 'pre-ind. low' , fontsize=16, color='C3')
ax.text(2350, 14, 'HadISST'      , fontsize=16, color='C4')

plt.tight_layout()
plt.savefig(f'{path_results}/SST/SOM_overview_raw')

detrended

In [ ]:
%%time
SOM_cheb13_dt_ctrl = chebychev(SOM_ctrl,13) - xr_lintrend(chebychev(SOM_ctrl,13))
SOM_cheb13_dt_rcp  = chebychev(SOM_rcp ,13) - xr_lintrend(chebychev(SOM_rcp ,13))
SOM_cheb13_dt_lpd  = chebychev(SOM_lpd ,13) - xr_lintrend(chebychev(SOM_lpd ,13))
SOM_cheb13_dt_lpi  = chebychev(SOM_lpi ,13) - xr_lintrend(chebychev(SOM_lpi ,13))
SOM_cheb13_dt_had  = chebychev(SOM_had ,13) - xr_lintrend(chebychev(SOM_had ,13))

SOM_cheb13_dt_ctrl.to_netcdf(f'{path_samoc}/SST/SOM_cheb13_index_ctrl.nc')
SOM_cheb13_dt_rcp .to_netcdf(f'{path_samoc}/SST/SOM_cheb13_index_rcp.nc' )
SOM_cheb13_dt_lpd .to_netcdf(f'{path_samoc}/SST/SOM_cheb13_index_lpd.nc' )
SOM_cheb13_dt_lpi .to_netcdf(f'{path_samoc}/SST/SOM_cheb13_index_lpi.nc' )
SOM_cheb13_dt_had .to_netcdf(f'{path_samoc}/SST/SOM_cheb13_index_had.nc' )


In [ ]:
SOM_cheb13_dt_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SOM_cheb13_index_ctrl.nc', decode_times=False)
SOM_cheb13_dt_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SOM_cheb13_index_rcp.nc' , decode_times=False)
SOM_cheb13_dt_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SOM_cheb13_index_lpd.nc' , decode_times=False)
SOM_cheb13_dt_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SOM_cheb13_index_lpi.nc' , decode_times=False)
SOM_cheb13_dt_had  = xr.open_dataarray(f'{path_samoc}/SST/SOM_cheb13_index_had.nc' , decode_times=False)


In [ ]:
SOM_cheb13_dt_ctrl

In [ ]:
# SOM_ctrl.plot()
# chebychev(SOM_ctrl,13).plot()
SOM_cheb13_dt_ctrl.plot()
# xr_lintrend(chebychev(SOM_ctrl,13)).plot()
# (chebychev(SOM_ctrl,13) - xr_lintrend(chebychev(SOM_ctrl,13))).plot()

In [ ]:
f,ax = plt.subplots(1,1,figsize=(12,5), sharex=True)

ax.tick_params(labelsize=14)
ax.set_xlabel('time [years]', fontsize=16)

ax.text(1950, .3, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .3, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .3, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .3, 'pre-ind. low' , fontsize=16, color='C3')


ax.set_ylabel('lin. detr. SOM index [$^\circ$C]', fontsize=16)
ax.axhline(0, c='k', lw=.5)
ax.plot(SOM_ctrl.time/365+1850, SOM_cheb13_dt_ctrl.values, c='C0', ls='-', label='chebychev 13 yr')
ax.plot(SOM_rcp .time/365+ 200, SOM_cheb13_dt_rcp .values, c='C1', ls='-')
ax.plot(SOM_lpd .time/365+1350, SOM_cheb13_dt_lpd .values, c='C2', ls='-')
ax.plot(SOM_lpi .time/365-1600, SOM_cheb13_dt_lpi .values, c='C3', ls='-')
ax.plot(SOM_had .time/365+2350, SOM_cheb13_dt_had .values, c='C4', ls='-')

ax.legend(fontsize=14, ncol=2, frameon=False, loc=4)
ax.set_xticks(np.arange(0,2400,200))

plt.tight_layout()
plt.savefig(f'{path_results}/SST/SOM_overview')

## spectra

In [ ]:
SOMs = [SOM_ctrl, SOM_rcp, SOM_lpd, SOM_lpi, SOM_had]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (5):
    label= ['CTRL', 'RCP', 'LPD', 'LPI', 'Had'][i]
    f, Pxx = sp.signal.welch(SOMs[i]-xr_lintrend(SOMs[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()